In [1]:
library(scds)
library(SingleCellExperiment)
library(DropletUtils)
library(Seurat)
library(Matrix)

ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F)
options(repr.plot.width=11, repr.plot.height=8.5)

Loading required package: SummarizedExperiment

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.ma

In [9]:
# read in srna-seq matrix files, these folders should have a barcodes.tsv, genes.tsv, and matrix.mtx files
STARsoloDIR= '../../../data/raw_data/STARsolo_out'
STARsolo_fn = file.path(STARsoloDIR, c("London_DLPFC-1.Solo.out", "Memphis_DLPFC-1.Solo.out"), 'GeneFull/filtered')
names(STARsolo_fn) = c("London_DLPFC-1", "Memphis_DLPFC-1")
sce <- read10xCounts(STARsolo_fn, version = '3')

# Infer doublets w/ SCDS algorithm

In [12]:
sce <- cxds_bcds_hybrid(sce,list("retRes"=TRUE))

[08:25:03] WARNING: amalgamation/../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [14]:
colData(sce)
table(ifelse(colData(sce)$hybrid_score < 1.0, 'OK cell', 'inferred doublet'))

DataFrame with 33841 rows and 5 columns
               Sample          Barcode       cxds_score           bcds_score
          <character>      <character>        <numeric>            <numeric>
1      London_DLPFC-1 AAACAGCCAACCGCCA 43378.3603333041  0.00318628735840321
2      London_DLPFC-1 AAACAGCCACTGGCTG 269062.923709876   0.0493841283023357
3      London_DLPFC-1 AAACAGCCACTTAACG 659909.246677727    0.955575585365295
4      London_DLPFC-1 AAACAGCCAGGACCAA 170211.659049272  0.00540034472942352
5      London_DLPFC-1 AAACAGCCAGGCGAGT 24040.7114497954  0.00221745599992573
...               ...              ...              ...                  ...
33837 Memphis_DLPFC-1 TTTGTTGGTATTTGGG 87683.5878403367 0.000911889248527586
33838 Memphis_DLPFC-1 TTTGTTGGTGAAACAA  93974.168596278  0.00469867093488574
33839 Memphis_DLPFC-1 TTTGTTGGTGACCTGG 48576.4302630521  0.00168665987439454
33840 Memphis_DLPFC-1 TTTGTTGGTGATGAGG 35635.8516505646 0.000922617386095226
33841 Memphis_DLPFC-1 TTTGTTGGTTGCAT


inferred doublet          OK cell 
            3030            30811 

In [17]:
sce = sce[, colData(sce)$hybrid_score < 1.0 ]
sce_fn = file.path('../../../data/tidy_data/rdas', 'raw_multiomeRNA_DLPFC_sce.rds')
saveRDS(sce, sce_fn)